In [1]:
# Author : Paul-Antoine Nguyen

# This script considers all the products a user has ordered
#
# We train a model computing the probability of reorder on the "train" data
#
# For the submission, we keep the orders that have a probability of
# reorder higher than a threshold


import numpy as np
import pandas as pd
import lightgbm as lgb
IDIR = '../input/'

In [2]:
print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
trains = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(IDIR + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(trains.shape, ', '.join(trains.columns)))


loading prior
loading train
loading orders
loading products
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered


In [3]:
priors = pd.merge(priors, orders, on="order_id", how="inner")

In [4]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


## Order product features

In [ ]:
FEATURES_PATH = './features2/'

In [7]:
user_info = orders[orders.eval_set == "prior"].groupby('user_id') \
        .agg({'order_number': np.max}) \
        .rename(columns={'order_number': 'user_orders'})
        
user_info.reset_index(inplace = True)

In [25]:
user_info.head()

,user_id,user_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [9]:
up_info = pd.DataFrame(priors.groupby(['user_id', 'product_id']).size().astype(np.int16), columns=['up_orders'])
up_info['up_first_order', 'up_last_order'] = priors.groupby(['user_id', 'product_id'].order_number.agg([np.min, np.max])
up_info['up_average_cart_order'] = (priors.groupby(['user_id', 'product_id']).add_to_cart_order.mean()).astype(np.float32)

up_info.reset_index(inplace = True)
up_info.user_id = up_info.user_id.astype(np.int32)
up_info.product_id  = up_info.product_id.astype(np.int32)

In [ ]:
up_info = pd.merge(up_info, user_info, on='user_id', how='left')
up_info['up_order_rate'] = (up_info.up_orders/up_info.user_orders).astype(np.float32)
up_info['up_orders_since_last_order'] = (up_info.user_orders - up_info.up_last_order).astype(np.float32)
up_info['up_order_rate_since_first_order'] = (up_info.up_orders / (up_info.user_orders - up_info.up_first_order + 1)).astype(np.float32)

In [16]:
up_info.to_csv(FEATURES_PATH + "up_info.csv")

In [60]:
up_info.head()

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_order,user_orders,up_orders_since_last_order,up_order_rate_since_first_order,up_order_rate
0,1,196,10,1,10,1.400000,10,0.0,1.000000,1.0
1,1,10258,9,2,10,3.333333,10,0.0,1.000000,0.9
2,1,10326,1,5,5,5.000000,10,5.0,0.166667,0.1
3,1,12427,10,1,10,3.300000,10,0.0,1.000000,1.0
4,1,13032,3,2,10,6.333333,10,0.0,0.333333,0.3


In [61]:
up_info.memory_usage().sum()/1000000

532.31812